# Equal-Weight S&P 500 Index Fund

## Library Imports

The first thing we need to do is import the open-source software libraries.

In [ ]:
!pip install XlsxWriter # installs xlsxwriter
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
"""All libraries need to be imported"""

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 1.9 MB/s eta 0:00:00


'All libraries need to be imported'

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Now it's time to import these stocks to our Jupyter Notebook file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Use the appropriate path to the 'sp_500_stocks.csv' file in your Google Drive
file_path = '/content/drive/MyDrive/sp_500_stocks.csv'
stocks = pd.read_csv(file_path)
stocks

Mounted at /content/drive


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.


In [ ]:
secrets_file_path = '/content/drive/MyDrive/my_secrets.py'
exec(open(secrets_file_path).read())
print(IEX_CLOUD_API_TOKEN)

Tpk_059b97af715d417d9f49f50b51b1c448


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud.

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [ ]:
symbol = 'AAPL'
api_token = "sk_100dbb99447c4f4b9607ca9cbaa4ba95"  # Replace with your actual IEX Cloud API token

api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={api_token}'

response = requests.get(api_url)

# Check the response status code
if response.status_code == 200:
    # If the response status code is 200, it means the request was successful
    data = response.json()
    print(data)
else:
    # If the response status code is not 200, there might be an error
    print(f"API request failed with status code: {response.status_code}")


{'avgTotalVolume': 53322608, 'calculationPrice': 'close', 'change': -1.19, 'changePercent': -0.00616, 'close': 191.94, 'closeSource': 'official', 'closeTime': 1689969600183, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 191.94, 'delayedPriceTime': 1689969599493, 'extendedChange': 0.11, 'extendedChangePercent': 0.00057, 'extendedPrice': 192.05, 'extendedPriceTime': 1689983991737, 'high': 194.97, 'highSource': '15 minute delayed price', 'highTime': 1689969632977, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 192.22, 'iexBidSize': 102, 'iexClose': 191.94, 'iexCloseTime': 1689969599868, 'iexLastUpdated': 1689969599868, 'iexMarketPercent': 0.011289770664572225, 'iexOpen': 194.09, 'iexOpenTime': 1689946200122, 'iexRealtimePrice': 191.94, 'iexRealtimeSize': 300, 'iexVolume': 812318, 'lastTradeTime': 1689969599902, 'latestPrice': 191.94, 'latestSource': 'Close', 'latestTime': 'July 21, 2023', 'latestUpdate': 1689969600183, 'latestVolume': 71951683, 'low': 191.23, 'lowSourc

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy.

With that said, the data isn't in a proper format yet. We need to parse it first.

In [ ]:
price = data['latestPrice']
print(price)
marketcap = data['marketCap']
print(marketcap)

191.94
3018967061880


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [ ]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [ ]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            marketcap,
            'N/A'
        ]
        , index = my_columns
        ), ignore_index=True
)

<ipython-input-8-fb3baa209822>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,191.94,3018967061880,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_token = "sk_100dbb99447c4f4b9607ca9cbaa4ba95"  # Replace with your actual IEX Cloud API token
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={api_token}'
    response = requests.get(api_url)
    data = response.json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index = my_columns),
        ignore_index = True
    )

<ipython-input-9-0740d95f84a4>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-9-0740d95f84a4>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-9-0740d95f84a4>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-9-0740d95f84a4>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-9-0740d95f84a4>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

In [ ]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.62,37695861893,N/A
1,AAL,17.42,11381573635,N/A
2,AAP,70.73,4204463652,N/A
3,AAPL,191.94,3018967061880,N/A
4,ABBV,143.74,253598998603,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [ ]:
def chunks(lst, n):
  for i in range(0, len(lst), n):
    yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={api_token}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        if symbol in data:
            final_dataframe = final_dataframe.append(
                pd.Series([
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index=my_columns),
                ignore_index=True
            )
        else:
            print(f"Data not found for symbol: {symbol}")

final_dataframe

<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

Data not found for symbol: DISCA


<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

Data not found for symbol: HFC


<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

Data not found for symbol: VIAC
Data not found for symbol: WLTW


<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-12-1d0885a00179>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.62,37695861893,N/A
1,AAL,17.42,11381573635,N/A
2,AAP,70.73,4204463652,N/A
3,AAPL,191.94,3018967061880,N/A
4,ABBV,143.74,253598998603,N/A
...,...,...,...,...
496,YUM,137.54,38523163642,N/A
497,ZBH,142.23,29664809406,N/A
498,ZBRA,306.46,15761341383,N/A
499,ZION,36.11,5347916313,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [ ]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")


Enter the value of your portfolio:10000000


In [ ]:
position_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.62,37695861893,156
1,AAL,17.42,11381573635,1145
2,AAP,70.73,4204463652,282
3,AAPL,191.94,3018967061880,103
4,ABBV,143.74,253598998603,138
...,...,...,...,...
496,YUM,137.54,38523163642,145
497,ZBH,142.23,29664809406,140
498,ZBRA,306.46,15761341383,65
499,ZION,36.11,5347916313,552


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [ ]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [ ]:
writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
writer.save()

<ipython-input-17-6e1d423bac1e>:5: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


This code works, but it violates the software principle of "Don't Repeat Yourself".

Let's simplify this by putting it in 2 loops:

In [ ]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Marget Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format],
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()

<ipython-input-19-16d93d5523a4>:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/usr/local/lib/python3.10/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
